```{=html}
<!--
## to run interactively in RStudio, set python install location
reticulate::use_python("C:/Users/Dale/AppData/Local/Programs/Python/Python310/python.exe")
-->
```

In [ ]:
#| label: py-interactive-setup
#| eval: false
#| echo: false

## to run interactively, set the working directory in python
import os
os.chdir("c:/Users/Dale/Desktop/fake-stroop")

In [ ]:
#| label: py-setup
from IPython.display import display, Markdown
import pandas as pd
import numpy as np
from scipy import stats as st 
import math
import os
import re
import sys

raw_data_subdir = "sample"
stroop_colours = set(['blue', 'brown', 'green', 'purple', 'red'])

## Data preprocessing

In [ ]:
## load in demographics
demo_raw = pd.read_csv(os.path.join(raw_data_subdir, "demographics.csv"))

## use regular expression to identify subject data files
rx = re.compile('^S[0-9]{2}\\.csv$')

files_todo = list(filter(rx.match, os.listdir(raw_data_subdir)))

## read in subject data files using a loop
li = []
for f in files_todo:
  df = pd.read_csv(os.path.join(raw_data_subdir, f))
  sub_id = re.sub("^S([0-9]{2})\\.csv$", "\\1", f)
  li.append(df.assign(id = int(sub_id, base = 10)))

trials_raw = pd.concat(li, axis = 0, ignore_index = True)

## move 'id' to first column
col1 = trials_raw.pop('id')
trials_raw.insert(0, 'id', col1)

## read in transcript
transcript_raw = pd.read_csv(os.path.join(raw_data_subdir, "transcript.csv"))

### Data cleaning

Now let's check any values that were entered by a human for typos and consistency.

#### `eng_lang` variable in `demo_raw`

In [ ]:
demo_raw['eng_lang'] = demo_raw['eng_lang'].str.lower()

## get rid of bogus ages
demo_raw.loc[(demo_raw['age'] < 16) | (demo_raw['age'] > 100), 'age'] = np.NaN

## replace 'nativ' with 'native'
demo_raw.loc[demo_raw['eng_lang'] == 'nativ', 'eng_lang'] = 'native'

el_vals = (
  demo_raw
  .dropna(subset = ['eng_lang'])
  .drop_duplicates(['eng_lang'])
)

el_vals['eng_lang']

if set(el_vals['eng_lang']) != set(['native', 'nonnative']):
  sys.exit("anomalous values found in 'eng_lang'")
  
demo = demo_raw

#### `response` variable in `transcript_raw`

In [ ]:
all_resp = set(transcript_raw['response'])

anom_vals = sorted(all_resp.difference(stroop_colours))
repaired = ['blue', 'brown', 'brown', 'brown', 'blue', 'green', 'green',
            'purple', 'purple', 'purple', 'purple', 'red']

transcript_raw['response'] = (transcript_raw['response']
                              .replace(anom_vals, repaired))

if set(transcript_raw['response']) != stroop_colours:
  sys.exit("anomalous values detected in transcript `response` column")

### Trial data

In [ ]:
tr = (trials_raw
  .query('event == "DISPLAY_ON"')
  .filter(['id', 'trial', 'data'])
)

tr[['stimword', 'inkcolour']] = tr['data'].str.split('-', 1, expand=True)

tr['inkcolour'] = tr['inkcolour'].apply(lambda x: re.sub("\\.png$", "", x))

tr['condition'] = 'incongruent'
tr.loc[tr['stimword'].str.lower() == tr['inkcolour'], 
       'condition'] = 'congruent'

tr2 = tr.drop('data', axis = 1)

tr_acc = pd.merge(tr2, transcript_raw, "left", on = ["id", "trial"])

trials_acc = (tr_acc
              .assign(is_accurate = tr_acc['response'] == tr_acc['inkcolour'])
              )

trials_rt = (trials_raw
  .drop('data', axis = 1)
  .pivot(index = ['id', 'trial'], columns = 'event', values = 'timestamp')
  .sort_index(level = [1, 0])
)

trials_rt['rt'] = trials_rt['VOICE_KEY'] - trials_rt['DISPLAY_ON']

tr4 = pd.merge(trials_acc, 
               trials_rt.drop(['DISPLAY_ON', 'VOICE_KEY'], axis = 1),
               on = ['id', 'trial'])

## perform a 'semi-join' on demo table excluding missing values for eng_lang
in_both = tr4['id'].isin(set(demo.query('eng_lang.notnull()')['id']))

trials = tr4[in_both]

n_trials_wrong = len(trials.query('is_accurate == False').index)
n_trials_NA = len(trials.query('rt.isnull()').index)

### Subject means

In [ ]:
cb = pd.merge(demo, trials, on = 'id')

sub_means = (cb
  .query('is_accurate')
  .groupby(['id', 'eng_lang', 'condition'])
  .agg({'rt' : 'mean'})
  .reset_index()
)

sub_means['rt'] = round(sub_means['rt']).astype('int64')

## this is now the 'idealised' data
sub_means_wide = (sub_means
  .pivot(index = ['id', 'eng_lang'], columns = 'condition', values = 'rt')
  .sort_values(['eng_lang', 'id'], ascending = True)
  .reset_index()
  .rename_axis(None, axis = 1)
  .drop('id', axis = 1)
)

sub_means_wide

## make sure we're getting the same subject means that we got from R
table_from_r = pd.read_csv("subject-means.csv")

if sub_means_wide.equals(table_from_r) == False:
  sys.exit("'sub_means_wide' did not match result from R")

## Compute statistics

In [ ]:
sub_effects = sub_means_wide.assign(effect = sub_means_wide['incongruent'] - 
                                      sub_means_wide['congruent'])

overall_stroop = sub_effects['effect']

result_1s = st.ttest_1samp(overall_stroop, popmean = 0)

faster_or_slower = "faster" if np.mean(overall_stroop) > 0 else "slower"

t_crit_1s = st.t.ppf(.975, 39)
stderr_1s = np.std(overall_stroop) / math.sqrt(len(overall_stroop) - 1)
o_mean = np.mean(overall_stroop)

ci_lower_1s = round(o_mean - t_crit_1s * stderr_1s)
ci_upper_1s = round(o_mean + t_crit_1s * stderr_1s)

native_N = sub_effects.value_counts('eng_lang').get('native')
nonnative_N = len(sub_effects) - native_N

grp_means = (sub_effects
             .groupby('eng_lang')
             .agg({'effect' : 'mean'})
             .reset_index())

grp_sd = (sub_effects
             .groupby('eng_lang')
             .agg({'effect' : 'std'})
             .reset_index())


native_eff = grp_means.at[0, 'effect']
nonnative_eff = grp_means.at[1, 'effect']

native_sd = grp_sd.at[0, 'effect']
nonnative_sd = grp_sd.at[1, 'effect']

neff = sub_effects.loc[sub_effects['eng_lang'] == 'native', 'effect']
nneff = sub_effects.loc[sub_effects['eng_lang'] == 'nonnative', 'effect']

ttest_grp = st.ttest_ind(neff, nneff)
sig_or_not = "was" if ttest_grp.pvalue < .05 else "was not"

sd_pooled_num = ((native_N - 1) * (native_sd ** 2) + 
                 (nonnative_N - 1) * (nonnative_sd ** 2))
                 
sd_pooled = math.sqrt(sd_pooled_num / (native_N + nonnative_N - 2))

cohen_d = abs(native_eff - nonnative_eff) / sd_pooled

std_err = math.sqrt( (native_sd ** 2) / native_N +
                     (nonnative_sd ** 2) / nonnative_N )
                     
t_crit = st.t.ppf(.975, native_N + nonnative_N - 2)

ci_lower = (native_eff - nonnative_eff) - t_crit * std_err
ci_upper = (native_eff - nonnative_eff) + t_crit * std_err

## Results

::: {#results style="background-color: rgb(220, 220, 255); padding: 5px;"}

In [ ]:
#| echo: false

## define some helper functions for displaying text
def ul(s, delimit = True):
  if delimit:
    dtr = "$"
  else:
    dtr = ""
  is_text = False
  if (isinstance(s, str)):
    is_text = re.sub("\s", "", s).isalpha()
  if is_text:
    result = dtr + "\\underline{\\text{" + s + "}}" + dtr
  else:
    result = dtr + "\\underline{" + str(s) + "}" + dtr
  return result

def perc(num, den, digits = 0):
  return round(100 * num / den, digits)

## deal with small numbers
def si(x):
  if (x <= 10):
    num_lookup = {
      "1" : "one",
      "2" : "two",
      "3" : "three",
      "4" : "four",
      "5" : "five",
      "6" : "six",
      "7" : "seven",
      "8" : "eight",
      "9" : "nine",
      "10" : "ten"
    }
    result = num_lookup.get(str(x))
  else:
    result = str(x)
  return result

def pvalue_str(pval):
  if (pval < .001):
    result = "< .001"
  else:
    result = "=" + str(round(pval, 3))
  return result

ngood = len(cb.query('is_accurate & rt.notnull()').index)

display(Markdown("""
We ran {npart} participants on a five-colour Stroop task.
""".format(npart = ul(len(demo.index)))))

display(Markdown("""
We had to remove data from {badnl} participants whose native language was not properly recorded by the experimenter. From the full set of {nall} trials recorded for the remaining participants, we removed {nwrong} trials ({pwrong}%) where participants produced the wrong answer and {nt_NA} ({pt_NA}%) further trials that could not be analysed because of voice key failure. This left {ngood} trials for analysis. For each participant, we calculated the mean response time in the congruent and incongruent condition.
""".format(badnl = ul(si(len(demo.query('eng_lang.isnull()').index))),
           nall = ul(f'{len(cb.index):,}'),
           nwrong = ul(si(len(cb.query('is_accurate == False')))),
           pwrong = ul(perc(len(cb.query('is_accurate == False')),
                            len(cb.index))),
           nt_NA = ul(si(len(cb.query('rt.isnull()').index))),
           pt_NA = ul(perc(len(cb.query('rt.isnull()').index),
                           len(cb.index))),
           ngood = ul(f'{ngood:,}')
           )))

display(Markdown("""
On average, speakers responded {o_mean} milliseconds (SD = {o_sd}) {faster_or_slower} in the congruent than in the incongruent condition, $t({t_df}) = {t_stat}$, $p {pstr}$, $d = {d_eff}$, 95% CI $[{ci_ll}, {ci_ul}]$.
""".format(o_mean = ul(round(np.mean(overall_stroop))),
           o_sd = ul(round(np.std(overall_stroop))),
           faster_or_slower = ul(faster_or_slower),
           t_df = ul(len(overall_stroop) - 1, False),
           t_stat = ul(round(result_1s.statistic, 2), False),
           pstr = ul(pvalue_str(result_1s.pvalue), False),
           d_eff = ul(round(np.mean(overall_stroop) /
                            np.std(overall_stroop), 2), False),
           ci_ll = ul(ci_lower_1s, False),
           ci_ul = ul(ci_upper_1s, False)
           )))

display(Markdown("""
Native English speakers (N = {native_N}) showed an average Stroop effect of {native_eff} ms (SD = {native_sd}), compared to an average of {nonnative_eff} ms (SD = {nonnative_sd}), for non-native English speakers (N = {nonnative_N}). According to a two-tailed independent-samples $t$-test with $\\alpha = .05$, the group difference {sig_or_not} statistically significant, $t({t_df})={t_stat}$, $p {t_pval}$, $d = {group_d}$, 95% CI $[{ci_lower}, {ci_upper}]$.
""".format(native_N = ul(native_N),
           native_eff = ul(round(native_eff)),
           native_sd = ul(round(native_sd)),
           nonnative_eff = ul(round(nonnative_eff)),
           nonnative_sd = ul(round(nonnative_sd)),
           nonnative_N = ul(nonnative_N),
           sig_or_not = ul(sig_or_not),
           t_df = ul(native_N + nonnative_N - 2, False),
           t_stat = ul(round(ttest_grp.statistic, 2), False),
           t_pval = ul(pvalue_str(ttest_grp.pvalue), False),
           group_d = ul(round(cohen_d, 2), False),
           ci_lower = ul(round(ci_lower), False),
           ci_upper = ul(round(ci_upper), False)
)))

:::